<img src="pic/icon_exercise_header_p1.png" width="100%"> <br>
# ICON Community Interface ComIn - Practical Exercise Notebooks  
**Setting Up Your Exercise Materials:** The practical exercises require several Jupyter notebooks, run scripts, ICON namelists, etc. All of them can be found in the `comin_training_exercise` directory. Therefore, please make sure that now you have a subdirectory named `comin_training_exercise` in your `home` directory.
# <br> Exercise P1: Programming a Rather Simple ComIn Python Plugin

---
 The **ICON Community Interface (ComIn)** enables the integration of **plugins** which is a flexible way to extend the code with your own routines and variables at run-time.  
ComIn offers the opportunity to write your plugins in various programming languages, including Fortran, C, and Python. In this training course, we will focus on developing **Python plugins**.
* **Exercise P1**: We learn how to allocate new ICON variables, using only a few lines of Python code.


**General Remarks:**
* For exercises, refer to Section 9.5 of the ICON tutorial [[1]](icon_exercise_programming.ipynb#Further-Reading-and-Resources) for the necessary information. In addition, the ICON ComIn project website contains a documentation with more detailed information [[2]](icon_exercise_programming.ipynb#Further-Reading-and-Resources).  
* To complete the following exercises, it's important to have a basic understanding of programming in Python (particularly the correct use of indentation).

In [ ]:
export SCRATCHDIR=/scratch/${USER::1}/$USER

<div class="alert alert-danger">
<b>Important note: The entire compilation process takes quite a few minutes. For this practical exercise, we can shorten this step by using a pre-compiled ICON model:</b>
</div>

In [ ]:
( cd ${SCRATCHDIR}
  tar xzf /pool/data/ICON/ICON_training/icon-2024.07-precompiled-levante.tar.gz )

Alternatively, we may download and extract the release tarball for *ICON-2024.07* as follows:

In [ ]:
( cd ${SCRATCHDIR}
  wget -qO- https://gitlab.dkrz.de/icon/icon-model/-/archive/icon-2024.07-public/icon-model-icon-2024.07-public.tar.gz | tar --transform 's/icon-model-icon-2024.07-public/icon/' -xz )

####  

### ICON Build Process

In [ ]:
export ICONDIR=${SCRATCHDIR}/icon

<div class="alert alert-success">
    <b style="color:#2d4b9b;">Exercise:</b> 
    In your local copy of the ICON code, create a directory <code>build</code> and change to this directory.
    Execute the configure wrapper for the "Levante" platform with the <code>gcc</code> compiler (<code>config/dkrz/levante.gcc</code>).<p/>
    <b>Important:</b> Please execute the configure wrapper with the configure option <code>--enable-comin</code>.</b>
    <br/><br/>
    Execute <code>make</code> with 4 processes.
    <br/>
    Due to the large amount of log output during the configuration and compiling, we highly recommend <b>performing this exercise in a terminal window</b>. <p/> <p/><button data-commandlinker-command="terminal:create-new">Create Terminal</button>
</div>

#### Solution

In [ ]:
export ICONDIR=${SCRATCHDIR}/icon
mkdir -p $ICONDIR/build
cd $ICONDIR/build
../config/dkrz/levante.gcc -q --enable-comin

In [ ]:
make -j4 2>&1 > compile.log

### Preliminary Step: Building the ICON ComIn Python Adapter

As you are writing a Python plugin in this exercise, there is no need to recompile the ICON code at any stage to complete the task.
However, we need to build the ICON ComIn Python Adapter. This is a task that only needs to be carried out once.

In [ ]:
source $HOME/comin-training-exercises/exercise/prepared/spack_initialize.sh
cd $ICONDIR/build
spack load py-mpi4py
(cd externals/comin/build && cmake -DCOMIN_ENABLE_EXAMPLES=ON  -DCOMIN_ENABLE_PYTHON_ADAPTER=ON .)
(cd externals/comin/build && make)

### Your First ComIn Plugin

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
  In this step, we need a Python script to serve as our ComIn plugin. You can access this Python script by clicking here:
    <p style="text-align: center"></center><a href="scripts/comin_plugin.py"><code style="color:#2d4b9b">scripts/comin_plugin.py</code></a>.</p>  <p/>
  In your plugin, add a 2D diagnostic variable named <code>comin_process_id</code> for domain 1.
  
 As outlined in the ICON tutorial Section 9.5, adding a new variable in your ComIn plugin involves two steps:
 <ul>
  <li>First you need to <b>register a new variable</b> and set the required metadata for the new variable. <br/>
      You will need the API function <code>comin.var_request_add</code> for this.  
      To determine which API function to use for setting metadata, use the <a href="https://icon-comin.gitlab-pages.dkrz.de/comin/d7/de6/md__2builds_2icon-comin_2comin_2doc_2comin__python__api.html">Python API</a> and for a table of available metadata keys, refer to the <a href="https://icon-comin.gitlab-pages.dkrz.de/comin/#autotoc_md36">Developer documentation</a>.
  <li>Secondly, you should <b>access your registered variable</b> like any other ICON variable within the secondary constructor of your plugin. <br/>
      This happens with the API function <code>comin.var_get</code>.
      Furthermore, you will need to specify the entry point for your callback function. In this exercise, we will use the <code>EP_ATM_WRITE_OUTPUT_BEFORE</code> entry point. </li>   
  </ul>
</div>

<figure style="float:right;">
<img src="pic/entry_points.PNG" style="margin:0px 20px 20px 0px;" width="600">
</figure>
The figure on the right provides a schematic overview of the available entry points in the ICON source code.
<ul>
    <li>ComIn entry points are depicted in <font color="green">green</font>, where the location of the entry points refers to the ComIn library v0.1.</li>
    <li>The entry point <code>EP_ATM_WRITE_OUTPUT_BEFORE</code> is highlighted with a <font color="red">red</font> box.</li>
</ul>

####  Solution:

```python
import comin

jg = 1  # set the domain id

# request to register the variable
var_descriptor = ("comin_process_id", jg)
comin.var_request_add(var_descriptor, lmodexclusive=False)
comin.metadata_set(var_descriptor, zaxis_id = comin.COMIN_ZAXIS_2D)


@comin.register_callback(comin.EP_SECONDARY_CONSTRUCTOR)
def simple_python_constructor():
    global comin_process_id
    comin_process_id = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("comin_process_id", jg), 
                                     flag=comin.COMIN_FLAG_WRITE)
```

####  

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
Write a callback function in your plugin
    <p style="text-align: center"><a href="scripts/comin_plugin.py"><code style="color:#2d4b9b">scripts/comin_plugin.py</code></a></p></P7> and attach it to the <code>EP_ATM_WRITE_OUTPUT_BEFORE</code> entry point. 
This is done using the <code>comin.register_callback</code> function (a function decorator).
<p/>
In the callback function, fill the variable <code>comin_process_id</code> with the MPI rank of the processor. 
</div>

<figure style="float:right;">
<img src="pic/ICON-ComIn-MPI-Communicators.png" style="margin:0px 20px 20px 0px;" width="600">
</figure>

**Hints:**
* Refer to the Python API description for ICON ComIn [[3]](icon_exercise_programming.ipynb#Further-Reading-and-Resources) to learn how to obtain the MPI rank of the ICON model in your plugin.<br/>
  You should use ComIn's so-called **host MPI communicator** which is the MPI communicator that comprises all MPI tasks of the ICON simulation which are involved in the ComIn callbacks.
* If you want to add a log message to your plugin script, you should use <code>print("...", file=sys.stderr)</code> to avoid buffering.

* In the callback function, import the NumPy package and use `numpy.asarray` to convert a variable handle you accessed into a NumPy array.

Actually, the last point, this use of `np.asarray` is an important point to note:  
The `comin.var_get` call returns a handle to an ICON variable. However, since ICON contains fields with multiple time levels, the corresponding array pointer changes during the model run. Each time the plugin needs to access the array (pointer), it must be explicitly or implicitly converted to a NumPy array.

#### Solution:

```python
@comin.register_callback(comin.EP_ATM_WRITE_OUTPUT_BEFORE)
def simple_python_callbackfct():
    import numpy as np
    # print("simple_python_callbackfct called!", file=sys.stderr)
    comin_process_id_np = np.asarray(comin_process_id)
    comin_process_id_np[:] = comin.parallel_get_host_mpi_rank()
```

####  

### Run the ICON Model

Now that you've finished writing your plugin, the next step is to run the ICON model.   

Plugins are enabled via a special namelist `comin_nml`.
In order to load your plugin dynamically while ICON is running, execute the cell below to add a `comin_nml` namelist to the ICON namelist file. We set the `plugin_library = libpython_adapter.so` with its absolute path such that it can be found by ICON and the operating system.

*Note: Alternatively, the search path for `libpython_adapter.so` could be set through the environment variable `LD_LIBRARY_PATH`.*

```bash
&comin_nml
   plugin_list(1)%name           = "comin_plugin"
   plugin_list(1)%plugin_library = "$ICONDIR/build/externals/comin/build/plugins/python_adapter/libpython_adapter.so"
   plugin_list(1)%options        = "$SCRIPTDIR/comin_plugin.py"
/
```

Now you can run the ICON model.

<figure style="float:right;">
<img src="reference/P1_comin_plugin/domain_decomp_p1.png" width="600">
<figcaption style="text-align:center"><em>Reference solution for "Step P1: Writing a ComIn Python Plugin".</em> </figcaption>
</figure>

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
    <ul>
        <li>
        <b>There is no need to recompile the ICON code!</b>
        </li>
        <li>
        You can use the script <a href="scripts/icon_exercise_comin_run.ipynb"><code style="color:#2d4b9b">icon_exercise_comin_run.ipynb</code></a> to run ICON.  <br/>
        Customize your namelist to ensure that <code>comin_process_id</code> is included in the model output files for the first domain.  
        </li>
        <li>
        Use the <a href="scripts/exercise_comin_plot_P1.ipynb"><code style="color:#2d4b9b">exercise_comin_plot_P1.ipynb</code></a> to visualize <code>comin_process_id</code>.<br/>
        Your result should look like the plot on the right.
        </li>
    </ul>
</div>

---

## Further Reading and Resources

- [1] ICON Tutorial, Ch. 9: https://www.dwd.de/DE/leistungen/nwv_icon_tutorial/nwv_icon_tutorial.html
- [2] Documentation on the ICON ComIn project website: https://icon-comin.gitlab-pages.dkrz.de/comin/ 
- [3] Python API description for ICON ComIn: https://icon-comin.gitlab-pages.dkrz.de/comin/d7/de6/md__2builds_2icon-comin_2comin_2doc_2comin__python__api.html

---

*Author info: Deutscher Wetterdienst (DWD) 2024 ::  comin@icon-model.org. For a full list of contributors, see CONTRIBUTING in the root directory. License info: see LICENSE file.*